# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** I have used a CNN-RNN architecture, in which the CNN encoder is a pre-trained **ResNet-50** (with the last Fully-connected layer removed) with aditional batch normalization layer. The output of the CNN is encoded into a feature vector. This feature vector, once pre-processed, is used as an input to the RNN decoder. The idea behind using a CNN as an encoder is because they excel at preserving spatial information from the images by compressing them into smaller representation.

The choice of RNN decoder is LSTM (Long Short Term Memeory). Based on the input from the encoder, the job of the RNN is to decode the information from CNN and turn it into natural language. The decoder consists of a single layer of LSTM and a Fully-connected layer witha dropout probability of 0.3. Based on the above mentioned paper: Neural Image Caption Generator (NIC), I've selected the value of no.of **embedded layer** and **hidden layers** as 512. [Reference: https://arxiv.org/pdf/1411.4555.pdf] 

Values of other relevant Hyperparameters: <br>
**batch_size** = 50 (for faster implementation) <br>
**vocab_threshold** = 5 (this value of Vocab threshold is good enough.) <br>
**vocab_from_file** = True (True, as will be loading an existing vocab file) <br>


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I've kept the tranformation same as provided. Since we want the size of the images to be uniform, the `trainsform_train` does a good job in resizing and normalizing the images. Uses RandomFlipHorizontal and RandomCrop for data augmentation.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** The trainable parameters that I've selected are:

        params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.batch.parameters())

Since we are using a pre-trained encoder, we don't need to learn the parameters of our CNN, however, we made a change in our CNN architecture, i.e., replacing the last dense layer with a embedding layer. Hence, we only learn the parameters of the embedding layer. Since we are using batch normalization we'll also learn it's parameter as well because β and γ in batch normalization are themselves learnable parameters that are updated during network training. [reference: https://www.mathworks.com/help/deeplearning/ref/nnet.cnn.layer.batchnormalizationlayer.html]

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** The paper: *Neural Image Caption Generator* has mentiones that they are using stochastic gradient descent, however, I am using Adam optimizer as it performs better SGD and also helps in reducing the noise.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 50          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.batch.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.88s)
creating index...


  0%|          | 766/414113 [00:00<01:54, 3610.02it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:33<00:00, 4418.15it/s]
/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

A: torch.Size([50, 10])
B: torch.Size([50, 9, 512])
Epoch [1/1], Step [1/8283], Loss: 9.2209, Perplexity: 10106.2190A: torch.Size([50, 18])
B: torch.Size([50, 17, 512])
Epoch [1/1], Step [2/8283], Loss: 9.1239, Perplexity: 9172.2823A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [3/8283], Loss: 8.9687, Perplexity: 7853.5011A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [4/8283], Loss: 8.7809, Perplexity: 6508.5710A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [5/8283], Loss: 8.6330, Perplexity: 5613.7576A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [6/8283], Loss: 8.2469, Perplexity: 3815.9663A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [7/8283], Loss: 7.8183, Perplexity: 2485.6021A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [8/8283], Loss: 7.1154, Perplexity: 1230.8155A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [

Epoch [1/1], Step [142/8283], Loss: 3.5472, Perplexity: 34.7173A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [143/8283], Loss: 3.6292, Perplexity: 37.6833A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [144/8283], Loss: 3.7980, Perplexity: 44.6132A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [145/8283], Loss: 3.6877, Perplexity: 39.9543A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [146/8283], Loss: 3.5009, Perplexity: 33.1437A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [147/8283], Loss: 3.8571, Perplexity: 47.3270A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [148/8283], Loss: 3.4822, Perplexity: 32.5312A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [149/8283], Loss: 3.5778, Perplexity: 35.7939A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [150/8283], Loss: 3.5988, Perplexity: 36.5552A: torch.

Epoch [1/1], Step [213/8283], Loss: 3.2946, Perplexity: 26.9669A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [214/8283], Loss: 3.5841, Perplexity: 36.0193A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [215/8283], Loss: 3.4749, Perplexity: 32.2933A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [216/8283], Loss: 3.2417, Perplexity: 25.5777A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [217/8283], Loss: 3.6190, Perplexity: 37.2993A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [218/8283], Loss: 3.6250, Perplexity: 37.5254A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [219/8283], Loss: 3.3127, Perplexity: 27.4590A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [220/8283], Loss: 3.5353, Perplexity: 34.3053A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [221/8283], Loss: 3.3432, Perplexity: 28.3087A: torch.

Epoch [1/1], Step [284/8283], Loss: 3.4072, Perplexity: 30.1814A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [285/8283], Loss: 3.1530, Perplexity: 23.4066A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [286/8283], Loss: 3.3358, Perplexity: 28.1011A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [287/8283], Loss: 3.0812, Perplexity: 21.7856A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [288/8283], Loss: 3.5184, Perplexity: 33.7310A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [289/8283], Loss: 3.1988, Perplexity: 24.5042A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [290/8283], Loss: 3.4216, Perplexity: 30.6173A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [291/8283], Loss: 3.3545, Perplexity: 28.6313A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [292/8283], Loss: 2.9702, Perplexity: 19.4952A: torch.

Epoch [1/1], Step [355/8283], Loss: 3.5581, Perplexity: 35.0967A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [356/8283], Loss: 2.8583, Perplexity: 17.4318A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [357/8283], Loss: 3.1251, Perplexity: 22.7616A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [358/8283], Loss: 3.2274, Perplexity: 25.2128A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [359/8283], Loss: 3.1377, Perplexity: 23.0514A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [360/8283], Loss: 2.8452, Perplexity: 17.2049A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [361/8283], Loss: 3.0743, Perplexity: 21.6347A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [362/8283], Loss: 3.0167, Perplexity: 20.4239A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [363/8283], Loss: 2.8133, Perplexity: 16.6646A: torch.

...

Epoch [1/1], Step [7874/8283], Loss: 2.0293, Perplexity: 7.6091A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [7875/8283], Loss: 2.0954, Perplexity: 8.1289A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7876/8283], Loss: 2.2666, Perplexity: 9.6465A: torch.Size([50, 25])
B: torch.Size([50, 24, 512])
Epoch [1/1], Step [7877/8283], Loss: 3.0026, Perplexity: 20.1370A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [7878/8283], Loss: 2.0775, Perplexity: 7.9843A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7879/8283], Loss: 2.3644, Perplexity: 10.6374A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7880/8283], Loss: 2.1835, Perplexity: 8.8777A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [7881/8283], Loss: 2.0146, Perplexity: 7.4980A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [7882/8283], Loss: 2.1370, Perplexity: 8.4736A: torc

Epoch [1/1], Step [7945/8283], Loss: 1.7346, Perplexity: 5.6669A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [7946/8283], Loss: 1.9908, Perplexity: 7.3214A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7947/8283], Loss: 2.2064, Perplexity: 9.0829A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7948/8283], Loss: 2.0129, Perplexity: 7.4853A: torch.Size([50, 10])
B: torch.Size([50, 9, 512])
Epoch [1/1], Step [7949/8283], Loss: 2.3101, Perplexity: 10.0759A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [7950/8283], Loss: 2.1957, Perplexity: 8.9861A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7951/8283], Loss: 2.0327, Perplexity: 7.6346A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [7952/8283], Loss: 2.2456, Perplexity: 9.4464A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [7953/8283], Loss: 1.9869, Perplexity: 7.2932A: torch.

Epoch [1/1], Step [8016/8283], Loss: 2.2597, Perplexity: 9.5804A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8017/8283], Loss: 1.9167, Perplexity: 6.7982A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [8018/8283], Loss: 2.0371, Perplexity: 7.6683A: torch.Size([50, 10])
B: torch.Size([50, 9, 512])
Epoch [1/1], Step [8019/8283], Loss: 2.2781, Perplexity: 9.7584A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [8020/8283], Loss: 1.9635, Perplexity: 7.1242A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [8021/8283], Loss: 2.2375, Perplexity: 9.3700A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [8022/8283], Loss: 1.9724, Perplexity: 7.1878A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [8023/8283], Loss: 2.2186, Perplexity: 9.1946A: torch.Size([50, 18])
B: torch.Size([50, 17, 512])
Epoch [1/1], Step [8024/8283], Loss: 2.6493, Perplexity: 14.1447A: torch.

Epoch [1/1], Step [8087/8283], Loss: 2.1746, Perplexity: 8.7989A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [8088/8283], Loss: 2.2916, Perplexity: 9.8908A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8089/8283], Loss: 1.8710, Perplexity: 6.4950A: torch.Size([50, 14])
B: torch.Size([50, 13, 512])
Epoch [1/1], Step [8090/8283], Loss: 2.1271, Perplexity: 8.3905A: torch.Size([50, 16])
B: torch.Size([50, 15, 512])
Epoch [1/1], Step [8091/8283], Loss: 2.0941, Perplexity: 8.1182A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8092/8283], Loss: 1.7661, Perplexity: 5.8479A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [8093/8283], Loss: 1.8540, Perplexity: 6.3852A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [8094/8283], Loss: 2.0762, Perplexity: 7.9738A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8095/8283], Loss: 2.0626, Perplexity: 7.8667A: torch.

Epoch [1/1], Step [8228/8283], Loss: 2.0055, Perplexity: 7.4300A: torch.Size([50, 15])
B: torch.Size([50, 14, 512])
Epoch [1/1], Step [8229/8283], Loss: 2.1165, Perplexity: 8.3023A: torch.Size([50, 17])
B: torch.Size([50, 16, 512])
Epoch [1/1], Step [8230/8283], Loss: 2.4811, Perplexity: 11.9546A: torch.Size([50, 13])
B: torch.Size([50, 12, 512])
Epoch [1/1], Step [8231/8283], Loss: 2.0879, Perplexity: 8.0682A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [8232/8283], Loss: 2.1549, Perplexity: 8.6272A: torch.Size([50, 11])
B: torch.Size([50, 10, 512])
Epoch [1/1], Step [8233/8283], Loss: 1.9487, Perplexity: 7.0196A: torch.Size([50, 19])
B: torch.Size([50, 18, 512])
Epoch [1/1], Step [8234/8283], Loss: 2.6636, Perplexity: 14.3472A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8235/8283], Loss: 1.9744, Perplexity: 7.2020A: torch.Size([50, 12])
B: torch.Size([50, 11, 512])
Epoch [1/1], Step [8236/8283], Loss: 1.9317, Perplexity: 6.9015A: torc

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.